In [1]:
%run Package_AND_Parameter.ipynb

C:\Users\vita7\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [68]:
def AutoCorr(data, k=1):

    data = data.values
    m = np.mean(data)
    N = len(data)

    res = np.sum([(data[i + k] - m) * (data[i] - m)
                  for i in range(N - k)]) / np.sum([(data[i + k] - m)**(2)
                                                    for i in range(N - k)])
    return res


def ThresholdVol(data, thr, mode='nab_beq'):
    data = data.values

    return {
        'ab_beq': lambda x: len(x[abs(x) >= thr]) + 1,
        'ab_s': lambda x: len(x[abs(x) < thr]) + 1,
        'nab_beq': lambda x: len(x[x >= thr]) + 1,
        'nab_s': lambda x: len(x[x < thr]) + 1
    }[mode](data)


def Power(data, sfreq):

    data = data.values
    freq, psd = signal.welch(data, sfreq)

    return sum(psd)


def Mean_frequency(data, sfreq=100):

    data = data.values
    freq, psd = signal.welch(data, sfreq)

    return np.dot(freq, psd) / sum(psd)


def Spect_slop(data, sfreq=100):

    data = data.values
    freq, psd = signal.welch(data, sfreq)
    slope, intercept, r_value, p_value, slope_std_error = stats.linregress(
        freq, psd)

    return slope


def Pattern_detection(data, m=5, r=1):
    #data = data.values
    N = len(data)
    r = r * data.std()
    vectors_ = [data[i:i + m] for i in range(0, (N - m))]
    dim = len(vectors_)
    res = 0

    for vt in vectors_:
        n = 0
        for vm in vectors_:
            dis_m = max([abs(vt[i] - vm[i]) for i in range(m)])
            if (dis_m <= r):
                n = n + 1
        res = res + (-(n / dim) * np.log(n / dim))

    return res

# TEST CODE

df = pd.read_csv('test.csv')
df.head()